In [244]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/pyskip_bench/src/")
sys.path.insert(0, "../../../projects/pyskip_blox/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [245]:
import random
import numpy as np
import pyskip
import pyskip_3d
import ipywebrtc
from pyskip_blox import minecraft, colors
from pyskip import functional as F
from pythreejs import *
from IPython.display import display

In [246]:
%%time
# level = minecraft.PySkipMinecraftLevel.load("../data/sample_minecraft_world.pickle.gz")
level = minecraft.PySkipMinecraftLevel.load("../data/Hogwarts1.0.pickle.gz")

CPU times: user 560 ms, sys: 35 µs, total: 560 ms
Wall time: 560 ms


In [247]:
level.rle_compression_ratio() / level.sparse_compression_ratio()

8.895387248740652

In [258]:
%%time
t = level.megatensor().min(1).to(float)

CPU times: user 27.5 s, sys: 481 ms, total: 28 s
Wall time: 3.17 s


In [259]:
def blur(tensor):
    G_x = (1 / 64.0) * pyskip.Tensor.from_list(
        [[[1, 6, 15, 20, 15, 6, 1]]]
    ).to(float)
    G_y = (1 / 64.0) * pyskip.Tensor.from_list(
        [[[1], [6], [15], [20], [15], [6], [1]]]
    ).to(float)
    G_z = (1 / 64.0) * pyskip.Tensor.from_list(
        [[[1]], [[6]], [[15]], [[20]], [[15]], [[6]], [[1]]]
    ).to(float)
    tensor = F.conv_3d(tensor, G_x, padding=(3, 0, 0), fill=0.0)
    tensor = F.conv_3d(tensor, G_y, padding=(0, 3, 0), fill=0.0)  
    tensor = F.conv_3d(tensor, G_z, padding=(0, 0, 3), fill=0.0)
    return tensor

def sharpen(tensor):
    kernel = pyskip.Tensor((3,3,3), val=-1)
    kernel[1,1,1] = 26
    return F.conv_3d(tensor, kernel, padding=(3,3,3), fill=0.0)

In [260]:
%%time
for i in range(0):
    t = blur(t)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs


In [261]:
%%time
avg_kernel = pyskip.Tensor.from_list([[[1, 1], [1, 1]], [[1, 1], [1, 1]]]).to(float) / 8
density_lattice = F.conv_3d(
    t,
    avg_kernel,
    padding=1,
)

CPU times: user 2.23 s, sys: 174 ms, total: 2.41 s
Wall time: 216 ms


In [262]:
%%time
mesh = pyskip_3d.marching_cubes(density_lattice._tensor, 0.1)

CPU times: user 6.45 s, sys: 160 ms, total: 6.61 s
Wall time: 3.78 s


In [263]:
geometry = BufferGeometry(
        attributes={
            "position": BufferAttribute(
                np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "index": BufferAttribute(
                np.array(mesh.triangles, dtype='uint32'),
                normalized=False,
            ),
        },
    )

geometry.exec_three_obj_method("computeVertexNormals")

In [264]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(color="white"),
    position=[0, 0, 0],
)

In [265]:
js_mesh

Preview(child=Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(3498908, 3), dtype=…

In [266]:
"""
camera_pos = level.xyz_to_pyskip_col(level.dense_dimensions())
camera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)
up = [0, 0, 0]
up[level.column_order[1]] = 1
camera.up = tuple(up)
"""

'\ncamera_pos = level.xyz_to_pyskip_col(level.dense_dimensions())\ncamera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)\nup = [0, 0, 0]\nup[level.column_order[1]] = 1\ncamera.up = tuple(up)\n'

In [267]:
light = PointLight(
    color='#ffffff',
    position=level.xyz_to_pyskip_col((0, 256,0)), # tuple(e for e in level.dense_dimensions())),
)
scene = Scene(children=[js_mesh, camera, light], background="black")

renderer = Renderer(
    camera=camera, 
    background="white", 
    background_opacity=1,
    scene=scene,
    #controls=[OrbitControls(controlling=camera)],
    width=600,
    height=500,
)

display(renderer)

Renderer(background='white', camera=PerspectiveCamera(fov=20.0, position=(-579.760096095058, 33.81020282430734…